### Symbology Lookup
The following module simplifies the ability to map your identifiers (RIC, ISIN, CUSIP) to an Organization ID required by the Filings interface.

In [1]:
from refinitiv.data.delivery import endpoint_request
import warnings

In [ ]:
class SymbolLookup:
    """The SymbolLookup is a convenient helper interface to convert commonly used symbols such as ticker, isin, cusip, sedol and RIC to 
       the rganization permID used by the Filings service.
       
       To start:
       
       1. Create a Symbol Lookup object, specifying the symbol
          lookup = FilingsQuery(ticker="<TICKER>" | isin="<ISIN>" | cusip="<CUSIP>" | sedol="<CUSIP>" | ric="<RIC>") 
          
       2. Optionally specify the MIC If using the ticker. The MIC code is used to resolve the exchange of the ticker.  For example:
          lookup.mic("XNAS")  # Nasdaq
          
       3. Perform the conversion
          permid = lookup.convert()
    """
    # MIC code - optionally used when specifying the ticker
    def mic(self, mic):
        """Defines the Organization ID used to query across the Filings database
        """
        if mic is None: 
            raise NameError('MIC must exist')  
            
        self.__mic = mic
        return self
    
    # Convert the symbol
    def convert(self):
        identifier_type, value = self.__identifier()
        
        body = {}
        
        # Determine type request body
        if identifier_type == "ExchangeTicker" and not self.__mic is None:
            # Apply our request to include the MIC
            body = self.__body_ticker_params(value, self.__mic)
        else:
            body = self.__body_params(identifier_type, value)
        
        # Request with Body Parameter - Symbology Lookup
        request_definition = endpoint_request.Definition(
            method = rd.delivery.endpoint_request.RequestMethod.POST,
            url = 'https://api.refinitiv.com/discovery/symbology/v1/lookup',
            body_parameters = body
        )
        
        response = request_definition.get_data()
        if not response.is_success:
            raise Exception(f'Failed to convert {identifier_type} symbol: {value}. {response.http_status}')
            
        results = response.data.raw["data"][0]["output"]
        
        if len(results) == 0:
            raise Exception('Request failed to match symbol.  No match returned.')
            
        matches = []
        for match in results:
            matches.append(match['value'])
            
        if len(matches) > 1:
            warnings.formatwarning = self.__warning_on_one_line            
            warnings.warn(f'Found multiple matches: {matches}.  Selecting the first one')
            
        return matches[0]
    
    def __warning_on_one_line(self, message, category, filename, lineno, file=None, line=None):
        return ' %s:%s: %s:%s' % (filename, lineno, category.__name__, message)
    
    def __body_params(self, identifier_type, value):
        return {
            "from": [
                {
                    "identifierTypes": [identifier_type],
                    "values": [value]
                }
            ],
            "to": [
                {
                    "identifierTypes": ["PermID"],
                    "objectTypes": ["organization"]
                }
            ],
            "type": "auto"
        }
    
    def __body_ticker_params(self, ticker, mic):
        return {
            "from": [
                {
                    "values": [
                      {
                          "exchangeTicker": ticker,
                          "MIC": mic
                      }
                    ]
                }
            ],
            "to": [
                {
                    "identifierTypes": ["PermID"],
                    "objectTypes": ["organization"]
                }
            ],
            "reference": [
                "name", 
                "status",
                "classification"
            ],
            "type": "predefined",
            "route": "ExchangeTickerMicToOrganization"
        }
    
    def __identifier(self):
        if not self.__ticker is None:
            return ("ExchangeTicker", self.__ticker)
        if not self.__ric is None:
            return ("RIC", self.__ric)
        if not self.__isin is None:
            return ("Isin", self.__isin)
        if not self.__cusip is None:
            return ("Cusip", self.__cusip)
        if not self.__sedol is None:
            return ("Sedol", self.__sedol)
        raise NameError('At least one type must be specified')  
    
    # Instantiate a SearchBrowser object
    def __init__(self, ticker=None, ric=None, isin=None, cusip=None, sedol=None):
        self.__ticker = ticker
        self.__ric = ric
        self.__isin = isin
        self.__cusip = cusip
        self.__sedol = sedol
        self.__mic = None
